# SQLite: Multiples Tablas y relaciones entre estas
![](http://2.bp.blogspot.com/-oi9q5jYCT0c/VntfDWxK7UI/AAAAAAAAACI/Y9ZrEbzxj_U/s1600/br1.gif)

Una de las caracteristicas clave de los sistemas de base de datos relacionales (RDBMS) es la relacion que existe entre los registros de diferentes tablas; esto es, que las columnas de las diferentes tablas estan relacionadas entre si. Esto con la intención de evitar la duplicación de información en la base de datos.

Esto se lográ estableciendo "llaves foraneas" en ciertas columnas, que se comportan como llaves importadas privadas importadas de otras tablas. En el siguiente esquema se tienen tres tablas (login_history, web_user, site_post) en la organización de un sitio web, relacionados entre si.

![schema](https://www.databasestar.com/wp-content/uploads/2019/07/ERD-Website.png)

La tabla "web_user" registra a los usuarios de un servicio web, con los campos de nombre, correo, si es un usuario activo, etc, y un user_id unico para distingir este registro de los demas: esto es la llave primaria (PK).

Cada vez que un usuario se registra en el servicio web, se guarda este evento en la tabla "login_history". Como el usuario se va a registrar en múltiples oportunidades, no tiene sentido (desde el punto de vista de la economía del registro de datos) registrar todos los campos de usuario en la tabla "login_history"; solamente basta con registrar el tiempo del registro (login_datetime) y la identificación única del usuario (user_id).

Para mantener la integridad de los datos es necesario establecer una relación entre el campo user_id de la tabla "web_user" y el campo user_id de la tabla "login_history". Esto quiere decir que no exista un registro en la tabla "login_history" de un usuario que no exista en la tabla "web_user": solo se pueden registrar en el servicio web usuarios que esten dados de alta en la página web. Para esto se define que el campo user_id de la tabla "login_history" es la llave foranea o foreign key (FK), esto significa que es una llave primaria que pertenece a otra tabla, insertada localmente. Como existe una relación entre ambos registros, cuando se intente guardar un registro en "login_history", el motor de la base de datos verificará que el campo user_id exista en "web_user" antes de modificar la tabla "login_history".

Esta relación es del tipo de "uno a muchos", ya que un solo usuario (user_id) puede tener muchos ingresos en el sistema.

Por otro lado, en la tabla "site_post", cada post (en este supuesto servicio web de blog) le pertence a un usuario. Cada post tiene un post_id (PK) y un user_id (FK). Esta relación que existe entre estos registros permite hacer consultas, considerando las conexiones que existan entre estas.

## Construyendo tablas con PK y FK
Para construir tablas relacionadas entre si con sqlite3 se debe de definir las diferentes llaves y las referencias entre estas.

    # Para la tabla web_user:
    CREATE TABLE web_user (
            user_id INTEGER PRIMARY KEY, 
            first_name TEXT, 
            last_name TEXT,
            email_address TEXT, 
            active INTEGER, 
            signup_datetime TEXT, 
            profile_image_name TEXT
    )

    # Para la tabla login_history
    CREATE TABLE login_history (
            login_datetime TEXT, 
            user_id INTEGER, 
            FOREIGN KEY(user_id)
                REFERENCES web_user (user_id))

    # Para la tabla site_post
    CREATE TABLE site_post (
            post_id INTEGER PRIMARY KEY, 
            user_id INTEGER, 
            post_datetime TEXT, 
            post_text TEXT, 
            FOREIGN KEY(user_id) 
                REFERENCES web_user (user_id))

## SQL JOIN TABLE
Las relaciones entre las tablas permite mantener la integridad de la información, de tal forma que las consultas retornen información correcta. Sin embargo, el establecimiento de llaves foraneas no incide sobre la capacidad de buscar información que se encuentre en diferentes tablas. Para esto necesitamos utilizar la instrucción SQL JOIN:

    # Para buscar todos los post de un usuario considerando su cuenta de correo electronico
    user_email_address = "example@email.com"

    SELECT post_id, post_datetime, post_text  
    FROM site_post JOIN web_user 
    ON site_post.user_id = web_user.user_id
    WHERE web_user.email_address = user_email_address

El código anterior realiza una "intersección" de dos tablas, extrayendo los registros tanto en "site_post" y en "web_user" que tengan la misma información en el campo user_id en las dos tablas. Esto retornará un subregistro de datos con todos los campos de ambas tablas (FROM site_post JOIN web_user) según el criterio de intersección (ON) sobre la que se podrá buscar datos según una condición (WHERE). Se puede utilizar la nomencatura *tabla.campo* en caso de que el subregistro considere dos campos con el mismo nombre en dos tablas diferentes (como el site_post.user_id y web_user.user_id).

La potencia de JOIN se puede observar cuando se requiere conseguir registros de tablas diferentes:

    # Para buscar todos los post y nombres de los usuarios que publicaron en una fecha determinada
    fecha = "2020/10/12"

    SELECT web_user.last_name, web_user.first_name, site_post.post_text
    FROM web_user JOIN site_post 
    ON web_user.user_id == site_post.user_id
    WHERE site_post.post_datetime = fecha

Todos estos son ejemplos de INNER JOIN. Existen otros tipos de JOIN que escapan del alcance de este material, pero se resumen en la siguiente figura:

![JOINS](https://ingenieriadesoftware.es/wp-content/uploads/2018/07/sqljoin.jpeg)

## Funciones de agregación
En sqlite3 también están presenten funciones de agregacion, que retornan el resultado de aplicar una operacion que colecciona todos los resultados de una operación en un solo valor, por ejemplo, en lugar de retornar todos los items de una consulta, se pueden agregar los resultados y sumar el total para obtener el número de resultados. Soporta el modificador DISTINCT para solo considera en el proceso de agregación valores diferentes.

    # Retorna el numero de usuarios registrados en una fecha determinada
    SELECT COUNT(DISTINCT user_id) FROM login_history WHERE login_datetime = "2020/10/1"

    # Retorna el total facturado en el mes
    SELECT SUM(monto) FROM facturas WHERE mes = 3

    # Retorna los valores maximos y minimos de ventas en un mes
    SELECT MAX(monto_factura), MIN(monto_factura) FROM facturas WHERE mes = 3

    # Retorna el numero promedio de ventas en el mes
    SELECT AVG(monto_factura) FROM facturas WHERE mes = 3

## GROUP BY
La instrucción GROUP BY permite agrupar los resultados obtenidos de una consulta en funcion de uno de los campos, donde se tenga un valor repetido

    # Retorna el monto promedio facturado cada mes
    SELECT mes, AVG(monto_factura) FROM facturas GROUP BY mes

# Ejemplo: Analisis de la base de datos Chinook Database
Database URL: https://cdn.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
![Database Schema](https://cdn.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg)

In [6]:
import sqlite3
import requests
import zipfile
import os

## Descargue la tabla chinook de la fuente Web

In [14]:
URL = r"https://cdn.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip"
filename = URL.split("/")[-1]
r = requests.get(URL)

In [17]:
try:
    print(f"Descargando {filename} de {URL}")
    with open(filename, mode='wb') as file:
        file.write(r.content)
    
    print(f"Descomprimiendo archivos en .\\datafile")
    with zipfile.ZipFile(filename, mode='r') as zip_ref:
        zip_ref.extractall(".\\datafile")

except Exception as e:
    print(e)

Descargando chinook.zip de https://cdn.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip
Descomprimiendo archivos en .\datafile


## Inspeccionar la base de datos

In [28]:
database_file = ".\\datafile\\chinook.db"
conn = sqlite3.connect(database_file)

In [29]:
# CONSULTA DE TABLAS EN LA BASE DE DATOS (sqlite_master)
with conn:
    cur = conn.cursor()
    results = cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    
    print(f"Tablas en {database_file}")
    print("----------" + "-" * len(database_file))
    for item in results:
        print("-", item[0])

Tablas en .\datafile\chinook.db
-------------------------------
- albums
- sqlite_sequence
- artists
- customers
- employees
- genres
- invoices
- invoice_items
- media_types
- playlists
- playlist_track
- tracks
- sqlite_stat1


In [30]:
# INSPECCION DE UNA TABLA ESPECIFICA EN LA BASE DE DATOS
with conn:
    tabla = "invoices"
    cur = conn.cursor()
    results = cur.execute(f"SELECT * FROM {tabla}")
    
    print(f"Columnas en {tabla}")
    print("-----------" + "-" * len(tabla))
    for item in results.description:
        print("-", item[0])

Columnas en invoices
-------------------
- InvoiceId
- CustomerId
- InvoiceDate
- BillingAddress
- BillingCity
- BillingState
- BillingCountry
- BillingPostalCode
- Total


In [31]:
# SCRIPT DE INSPECCION DE UNA BASE DE DATOS (INSPECCION DE TABLA CON PRAGMA)
database_file = ".\\datafile\\chinook.db"
conn = sqlite3.connect(database_file)

with conn:
    cur = conn.cursor()
    results = cur.execute("SELECT name FROM sqlite_master WHERE type='table'")
    
    tablas_dict = {}
    
    print(f"Tablas en {database_file.upper()}")
    print("----------" + "-" * len(database_file))
    for idx, item in enumerate(results, start=1):
        print(f"[{idx}]: {item[0].upper()}")
        tablas_dict[idx] = item[0]
    else:
        tab_idx = int(input("Seleccione la tabla a inspeccionar: "))
        
        if tab_idx in tablas_dict:
            tabla = tablas_dict[tab_idx]
            #results = cur.execute(f"SELECT * FROM {tabla}")   Utilizando: results.description
            results = cur.execute(f"PRAGMA table_info ({tabla})")     # Utilizando PRAGMA
             
            print(f"\nColumnas en {tabla.upper()}")
            print("-----------" + "-" * len(tabla))
            for item in results:
                #print(f"  - {item[0]}")
                print(f" - {item[1]}: ({item[2]})")
                
        else:
            print("Opcion invalida")
    
        

Tablas en .\DATAFILE\CHINOOK.DB
-------------------------------
[1]: ALBUMS
[2]: SQLITE_SEQUENCE
[3]: ARTISTS
[4]: CUSTOMERS
[5]: EMPLOYEES
[6]: GENRES
[7]: INVOICES
[8]: INVOICE_ITEMS
[9]: MEDIA_TYPES
[10]: PLAYLISTS
[11]: PLAYLIST_TRACK
[12]: TRACKS
[13]: SQLITE_STAT1
Seleccione la tabla a inspeccionar: 4

Columnas en CUSTOMERS
--------------------
 - CustomerId: (INTEGER)
 - FirstName: (NVARCHAR(40))
 - LastName: (NVARCHAR(20))
 - Company: (NVARCHAR(80))
 - Address: (NVARCHAR(70))
 - City: (NVARCHAR(40))
 - State: (NVARCHAR(40))
 - Country: (NVARCHAR(40))
 - PostalCode: (NVARCHAR(10))
 - Phone: (NVARCHAR(24))
 - Fax: (NVARCHAR(24))
 - Email: (NVARCHAR(60))
 - SupportRepId: (INTEGER)


## Mineria de datos en la dB chinoook.db

In [32]:
# METALLICA ESTA EN LA BASE DE DATOS?
database_file = ".\\datafile\\chinook.db"
conn = sqlite3.connect(database_file)

with conn:
    cur = conn.cursor()
    sql = """SELECT ArtistId, name FROM artists WHERE name LIKE '%metallica%' ORDER BY name"""
    results = cur.execute(sql)
    
    for item in results:
        print(f"{item[0]}: {item[1]}")
        

50: Metallica


In [33]:
# QUE Y CUANTOS ALBUMS TIENE METALLICA EN LA TIENDA?
database_file = ".\\datafile\\chinook.db"
conn = sqlite3.connect(database_file)

with conn:
    cur = conn.cursor()
    parameters = "Metallica"
    
    sql = """SELECT artists.name, COUNT(albums.title) 
             FROM artists JOIN albums 
             ON artists.ArtistId = albums.ArtistId 
             WHERE artists.name = ?"""
    results = cur.execute(sql, (parameters,))
    item = results.fetchone()
    print(f"Albums Registrados de {item[0]}: {item[1]}")
    
    sql = """SELECT artists.name, albums.title
             FROM artists JOIN albums 
             ON artists.ArtistId = albums.ArtistId 
             WHERE artists.name = ?"""
    results = cur.execute(sql, (parameters,))
    
    for item in results:
        print(f"  - {item[0]}: {item[1]}")
        

Albums Registrados de Metallica: 10
  - Metallica: Garage Inc. (Disc 1)
  - Metallica: Black Album
  - Metallica: Garage Inc. (Disc 2)
  - Metallica: Kill 'Em All
  - Metallica: Load
  - Metallica: Master Of Puppets
  - Metallica: ReLoad
  - Metallica: Ride The Lightning
  - Metallica: St. Anger
  - Metallica: ...And Justice For All


In [34]:
# LISTADO DE ALBUMS Y TRACKS DE METALLICA
database_file = ".\\datafile\\chinook.db"
conn = sqlite3.connect(database_file)

with conn:
    cur = conn.cursor()
    parameters = "Metallica"
    
    sql = """SELECT albums.AlbumId, albums.Title
             FROM artists JOIN albums 
             ON artists.ArtistId = albums.ArtistId 
             WHERE artists.name = ?"""
    results = cur.execute(sql, (parameters,))
    albums_id = [(item[0], item[1]) for item in results]
    
    for title in albums_id:
        parameters = title[0]
        sql = "SELECT Name, UnitPrice FROM tracks WHERE AlbumId = ?"
        results = cur.execute(sql, (parameters,))
        
        print(title[1])
        for idx, track in enumerate(results, start=1):
            print(f"  {idx:2}: {track[0]} - (${track[1]:.2f})")
        else:
            print()
 

Garage Inc. (Disc 1)
   1: Free Speech For The Dumb - ($0.99)
   2: It's Electric - ($0.99)
   3: Sabbra Cadabra - ($0.99)
   4: Turn The Page - ($0.99)
   5: Die Die My Darling - ($0.99)
   6: Loverman - ($0.99)
   7: Mercyful Fate - ($0.99)
   8: Astronomy - ($0.99)
   9: Whiskey In The Jar - ($0.99)
  10: Tuesday's Gone - ($0.99)
  11: The More I See - ($0.99)

Black Album
   1: Enter Sandman - ($0.99)
   2: Sad But True - ($0.99)
   3: Holier Than Thou - ($0.99)
   4: The Unforgiven - ($0.99)
   5: Wherever I May Roam - ($0.99)
   6: Don't Tread On Me - ($0.99)
   7: Through The Never - ($0.99)
   8: Nothing Else Matters - ($0.99)
   9: Of Wolf And Man - ($0.99)
  10: The God That Failed - ($0.99)
  11: My Friend Of Misery - ($0.99)
  12: The Struggle Within - ($0.99)

Garage Inc. (Disc 2)
   1: Helpless - ($0.99)
   2: The Small Hours - ($0.99)
   3: The Wait - ($0.99)
   4: Crash Course In Brain Surgery - ($0.99)
   5: Last Caress/Green Hell - ($0.99)
   6: Am I Evil? - ($0.99)
 

In [35]:
# CUAL ES LA FACTURACION DE METALLICA EN LA TIENDA?
database_file = ".\\datafile\\chinook.db"
conn = sqlite3.connect(database_file)

with conn:
    cur = conn.cursor()
    parameters = "Metallica"
    sql = """SELECT artists.Name, AVG(invoices.Total), SUM(invoices.Total)
             FROM artists 
                 JOIN albums 
                 JOIN tracks
                 JOIN invoice_items
                 JOIN invoices
             ON artists.ArtistId = albums.ArtistId 
                 AND albums.AlbumId = tracks.AlbumId
                 AND tracks.TrackId = invoice_items.TrackId
                 AND invoice_items.invoiceId = invoices.invoiceId
             WHERE artists.name = ?"""
    results = cur.execute(sql, (parameters,))
    
    for item in results:
        print(f"{item[0]} :    Fact. Prom: {item[1]:.2f}USD    Total: {item[2]:.2f}USD")
        

Metallica :    Fact. Prom: 6.59USD    Total: 599.94USD


In [38]:
# QUE PAISES SON LOS QUE MAS CONSUMEN PRODUCTOS DE METALLICA?
database_file = ".\\datafile\\chinook.db"
conn = sqlite3.connect(database_file)

with conn:
    cur = conn.cursor()
    parameters = "Metallica"
    sql = """SELECT customers.Country, SUM(invoices.Total) , AVG(invoices.Total)
             FROM artists 
                 JOIN albums 
                 JOIN tracks
                 JOIN invoice_items
                 JOIN invoices
                 JOIN customers
             ON artists.ArtistId = albums.ArtistId 
                 AND albums.AlbumId = tracks.AlbumId
                 AND tracks.TrackId = invoice_items.TrackId
                 AND invoice_items.invoiceId = invoices.invoiceId
                 AND invoices.CustomerId = customers.CustomerId
             WHERE artists.name = ?
             GROUP BY customers.Country
             ORDER BY SUM(invoices.Total) DESC
             LIMIT 5"""
    results = cur.execute(sql, (parameters,))
    
    for item in results:
        print(f"{item[0]:16}: {item[1]:10.2f} USD   {item[2]:10.2f} USD")

USA             :     177.21 USD         6.56 USD
Canada          :     119.79 USD         7.99 USD
France          :      94.05 USD         9.40 USD
United Kingdom  :      85.14 USD         9.46 USD
Germany         :      53.46 USD         5.94 USD


In [39]:
# CUALES SON LOS TRACKS MAS POPULARES DE METALLICA?
database_file = ".\\datafile\\chinook.db"
conn = sqlite3.connect(database_file)

with conn:
    cur = conn.cursor()
    parameters = "Metallica"
    sql = """SELECT tracks.name, SUM(invoice_items.Quantity)
             FROM artists 
                 JOIN albums 
                 JOIN tracks
                 JOIN invoice_items
             ON artists.ArtistId = albums.ArtistId 
                 AND albums.AlbumId = tracks.AlbumId
                 AND tracks.TrackId = invoice_items.TrackId
             WHERE artists.name = ?
             GROUP BY tracks.name 
             ORDER BY SUM(invoice_items.Quantity) DESC
             LIMIT 10"""
    results = cur.execute(sql, (parameters,))
    
    for item in results:
        print(f"{item[0]}: {item[1]}")

Whiskey In The Jar: 2
The Unforgiven II: 2
The House Jack Built: 2
Stone Cold Crazy: 2
Shoot Me Again: 2
Phantom Lord: 2
Nothing Else Matters: 2
Motorbreath: 2
Leper Messiah: 2
For Whom The Bell Tolls: 2
